## Aethalometer® Model AE33

In [ ]:
from AE33_device import AE33_device

device = AE33_device()
device.read_path_file()
device.print_params()
device.write_path_file()

RUN =  1
IP =  192.168.1.62
Port =  8002
pathfile =  ./data/
MINID =  943370


### To do
1. Данные формата D класть в файл xls
2. дата, время, BB%, BC1; BC2; BC3; BC4; BC5; BC6; BC7; дата, время, писать в таблицу xls. таблица на год
3. График за 2 суток на одном графике всех 8 параметров, плавающий, пересчет после каждого считывания.
4. Какой-то еще график

In [ ]:
import sys
import socket
import time
from datetime import datetime
import os


class AE33_device:
    def __init__(self):
        self.MINID = 0
        self.MAXID = 0

        ## for data files
        self.yy = '0'        ##  year for filename of raw file
        self.mm = '0'        ## month for filename of raw file
        self.yy_D = '0'      ##  year for filename of D-file
        self.mm_D = '0'      ## month for filename of D-file 
        self.pathfile = ''   ## work directory name
        self.file_raw = None       ## file for raw data
        self.file_format_D = None  ## file for raw data
        self.file_header = ''

        self.run_mode = 0

        self.buff = ''
        self.info = ''
        self.IPname = '213.131.1.46'
        self.IsConnected = 1
        self.Port = 8002  ## port number
        self.sock = None  ## socket

        self.fill_header()


    def fill_header(self):
        self.file_header = "AETHALOMETER\nSerial number = AE33-S08-01006\nApplication version = 1.6.7.0\nNumber of channels = 7\n\nDate(yyyy/MM/dd); Time(hh:mm:ss); Timebase; RefCh1; Sen1Ch1; Sen2Ch1; RefCh2; Sen1Ch2; Sen2Ch2; RefCh3; Sen1Ch3; Sen2Ch3; RefCh4; Sen1Ch4; Sen2Ch4; RefCh5; Sen1Ch5; Sen2Ch5; RefCh6; Sen1Ch6; Sen2Ch6; RefCh7; Sen1Ch7; Sen2Ch7; Flow1; Flow2; FlowC; Pressure(Pa); Temperature(°C); BB(%); ContTemp; SupplyTemp; Status; ContStatus; DetectStatus; LedStatus; ValveStatus; LedTemp; BC11; BC12; BC1; BC21; BC22; BC2; BC31; BC32; BC3; BC41; BC42; BC4; BC51; BC52; BC5; BC61; BC62; BC6; BC71; BC72; BC7; K1; K2; K3; K4; K5; K6; K7; TapeAdvCount;\n\n\n"


    def read_path_file(self):
        # check file
        #print("read file")
        try:
            f = open("PATHFILES.CNF")
        except:
            print("Error!! No file PATHFILES.CNF\n\n")
            return -1
        
        params = [x.replace('\n','') for x in f.readlines() if x[0] != '#']
        f.close()
        #print(params)
        
        for param in params:
            if "RUN" in param:
                self.run_mode = int(param.split('=')[1])
            elif "IP" in param:
                self.IPname = param.split('=')[1].split()[0]
                self.Port   = int(param.split('=')[1].split()[1])
            elif "MINID" in param:
                self.MINID = int(param.split('=')[1])
                self.MAXID = self.MINID
            else:
                self.pathfile = param
                os.system("mkdir " + param)
    # \todo ПОПРАВИТЬ в конфигурацилонном файле СЛЕШИ В ИМЕНИ ДИРЕКТОРИИ  !!!   для ВИНДА


    def write_path_file(self):
        f = open("PATHFILES.CNF.bak", 'w')
        f.write("#\n")
        f.write("# Programm mode:\n")
        f.write("#   1 - for MAIN-menu,  0 - Auto RUN\n")
        f.write("#\n")
        f.write("RUN=" + str(self.run_mode) + "\n")
        f.write("#\n")
        f.write("# Directory for DATA:\n")
        f.write("#\n")
        f.write(self.pathfile + '\n')
        f.write("#\n")
        f.write("# AE33:   IP address and Port:\n")
        f.write("#\n")
        f.write("IP=" + self.IPname + '  ' + str(self.Port) +"\n")
        f.write("#\n")
        f.write("# AE33:  Last Records:\n")
        f.write("#\n")
        f.write("MINID=" + str(self.MAXID) + "\n")
        f.write("#\n")
        f.close()


    def print_params(self):
        print("RUN = ", self.run_mode)
        print("IP = ", self.IPname)
        print("Port = ", self.Port)
        print("pathfile = ", self.pathfile)
        print("MINID = ", self.MINID)


    def connect(self):
        if self.active == -1:
            return -1
        
        #socket.socket(family='AF_INET', type='SOCK_STREAM', proto=0, fileno=None)
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM | socket.SOCK_NONBLOCK);
        self.sock.connect((self.IPname, self.Port)) 
        ## \todo проверить, что связь установлена


    def unconnect(self):
        ##  socket.shutdown(self.sock, SHUT_RD|SHUT_WR)
        self.sock.close()


    def request(self, command, start, stop):
        if command == 'FETCH DATA':
            command += ' ' + str(start) + ' ' + str(stop)
        command += '\r\n'

        ## --- send command ---
        time.sleep(1)
        ##self.sock.send(bytes(command))
        bytes = self.sock.send(command.encode())
        ## \todo проверить, что все отправилось
        if bytes != len(command):
            print("request: Error in sending data!! ") 
        print('sent ', bytes, ' to socket')

        if "CLOSE" in command:
            return 1

        ## --- read data ---
        time.sleep(1)
        attempts = 0
        self.buff = self.sock.recv()
        while(len(buff) == 0 and attempts < 10):
            time.sleep(1)
            self.buff = self.sock.recv()
            attempts += 1
        if attempts >= 10:
            print("request: Error in receive")
            self.sock.unconnect()
            return 2
        
        if "MAXID" in command:
            self.MAXID = int(buff)
        if "MINID" in command:
            self.MINID = int(buff)
        if '?' in command:
            self.info = buff
        if "FETCH" in command:
            self.parse_raw_data()
        if "AE33" in command:
            if "AE33:D":
                self.parse_format_D_data()            
        return 0


    def parse_raw_data(self):
        if len(self.buff) < 10:
            return
        self.buff = self.buff.split("AE33>")
        for line in self.buff:
            if len(line) < 50:
                continue

            mm, dd, yy = line.split("|")[2][:10].split('/')
            if mm != self.mm or yy != self.yy:
                filename = '_'.join((yy, mm)) + '_AE33-S08-01006.raw'
                filename = self.pathfile + filename
                if self.file_raw:
                    self.file_raw.close()
                self.file_raw = open(filename, "a")
            self.file_raw.write(line)
            self.file_raw.write('\n')
            
        self.file_raw.flush()
        self.mm = mm
        self.yy = yy


    def parse_format_D_data(self):
        ## for test 
        #f = open("buffer.txt")
        #self.buff = f.read().split('\n')
        #f.close()
        #print(self.buff)

        ## main
        if len(self.buff) < 10:
            return

        lastmm, lastyy = '0', '0'
        filename = ''
        lastline = ''
        need_check = True
        dateformat = "%Y/%m/%d %H:%M:%S"
        for line in self.buff[::-1]:
            print(line)
            yy, mm, _ = line.split()[0].split('/')
            #print(yy, mm)

            # for first line or new file
            if mm != lastmm or yy != lastyy:
                filename = '_'.join((yy, mm)) + '_AE33-S08-01006.ddat'
                try:
                    ## file exists
                    f = open(filename, 'r')
                    lastline = f.readlines()[-1].split()
                    lasttime = lastline[0] + ' ' + lastline[1]
                    lasttime = datetime.strptime(lasttime, dateformat)
                    f.close()
                    lastmm = mm
                    lastyy = yy
                    need_check = True
                except:
                    ## no file
                    f = open(filename, 'a')        
                    f.write(self.file_header)
                    f.close()
                    lastline = []
                    need_check = False

            ## check line
            if need_check: # and len(lastline):
                print(line)
                nowtime  = line.split()[0] + ' ' + line.split()[1]
                print(nowtime)
                nowtime  = datetime.strptime(nowtime,  dateformat)
                print(nowtime - lasttime)
                if nowtime <= lasttime:
                    continue

            need_check = False

            ## write to file
            f = open(filename, 'a')
            f.write(line)
            f.write('\n')
            f.close()
 



device = AE33_device()
device.read_path_file()
device.print_params()
device.write_path_file()
#device.connect()
device.parse_format_D_data()
#device.sock.send(bytes('Hello, world', encoding = 'UTF-8'))  # отправляем сообщение
#device.unconnect()

RUN =  1
IP =  192.168.1.62
Port =  8002
pathfile =  ./data/
MINID =  943370


In [ ]:
#import time
#import datetime
from datetime import datetime

def fill_header():
    selffile_header = "AETHALOMETER\nSerial number = AE33-S08-01006\nApplication version = 1.6.7.0\nNumber of channels = 7\n\nDate(yyyy/MM/dd); Time(hh:mm:ss); Timebase; RefCh1; Sen1Ch1; Sen2Ch1; RefCh2; Sen1Ch2; Sen2Ch2; RefCh3; Sen1Ch3; Sen2Ch3; RefCh4; Sen1Ch4; Sen2Ch4; RefCh5; Sen1Ch5; Sen2Ch5; RefCh6; Sen1Ch6; Sen2Ch6; RefCh7; Sen1Ch7; Sen2Ch7; Flow1; Flow2; FlowC; Pressure(Pa); Temperature(°C); BB(%); ContTemp; SupplyTemp; Status; ContStatus; DetectStatus; LedStatus; ValveStatus; LedTemp; BC11; BC12; BC1; BC21; BC22; BC2; BC31; BC32; BC3; BC41; BC42; BC4; BC51; BC52; BC5; BC61; BC62; BC6; BC71; BC72; BC7; K1; K2; K3; K4; K5; K6; K7; TapeAdvCount;\n\n\n"
    return selffile_header


def selfparse_format_D_data():
    ## for test
    selffile_header = fill_header()
    selfmm_D='11'
    selfyy_D='2021'
    f = open("buffer.txt")
    selfbuff = f.read().split('\n')
    f.close()
    print(selfbuff)
    #print(len(selfbuff))

    # main
    lastmm, lastyy = '0', '0'
    filename = ''
    lastline = ''
    need_check = True
    dateformat = "%Y/%m/%d %H:%M:%S"
    for line in selfbuff[::-1]:
        print(line)
        yy, mm, _ = line.split()[0].split('/')
        #print(yy, mm)

        # for first line or new file
        if mm != lastmm or yy != lastyy:
            filename = '_'.join((yy, mm)) + '_AE33-S08-01006.ddat'
            try:
                ## file exists
                f = open(filename, 'r')
                lastline = f.readlines()[-1].split()
                lasttime = lastline[0] + ' ' + lastline[1]
                print("1==>", lasttime)
                lasttime = datetime.strptime(lasttime, dateformat)
                print("2==>", lasttime)
                f.close()
                print("3==>", lasttime)
                lastmm = mm
                lastyy = yy
                need_check = True
            except:
                ## no file
                print("exc1==>", lasttime)
                f = open(filename, 'a')        
                f.write(selffile_header)
                f.close()
                lastline = []
                need_check = False

        ## check line
        if need_check: # and len(lastline):
            print(line)
            nowtime  = line.split()[0] + ' ' + line.split()[1]
            print(nowtime)
            nowtime  = datetime.strptime(nowtime,  dateformat)
            print(nowtime - lasttime)
            if nowtime <= lasttime:
                continue

        need_check = False

        ## write to file
        f = open(filename, 'a')
        f.write(line)
        f.write('\n')
        f.close()


selfparse_format_D_data()
print("OK")

['2021/12/20 00:08:00 60 951871 240458 584057 932577 307374 631018 925452 341798 638565 924099 465324 767084 925583 465984 768139 819660 535818 814599 895035 559514 831270 3796 1205 5001 101325 21.11 13.7 30 39 0 0 10 10 00000 32 158 195 312 165 201 302 162 207 282 146 223 228 184 229 288 134 134 191 189 256 265 0.004313926 0.004950107 0.005439872 0.006022566 0.006020389 0.00666612 0.006853341 1008 6 0 0 21.6 16.5 -5.0 23.1 14.1 -5.9', '2021/12/20 00:07:00 60 951824 240412 583980 932491 307294 630926 925398 341734 638491 924003 465221 766978 925493 465894 768035 819564 535707 814466 894887 559384 831114 3796 1205 5001 101325 21.11 60.4 30 41 0 0 10 10 00000 32 115 211 229 174 175 317 147 207 258 152 140 238 139 175 218 174 281 249 140 147 196 0.004314291 0.004948705 0.005439783 0.006020301 0.006020041 0.006669421 0.006851992 1008 6 0 0 21.6 16.5 -5.0 23.1 14.1 -5.9', '2021/12/20 00:06:00 60 951840 240367 583938 932542 307264 630912 925414 341687 638464 924051 465203 766983 925507 46584

In [ ]:
def write_path_file():
    selfrun_mode = 3
    selfpathfile = "./data1/"
    selfIPname = '192.168.1.62'
    selfPort = 8002
    selfMAXID = 9999
    f = open("PATHFILES.CNF.bak", 'w')
    f.write("#\n")
    f.write("# Programm mode:\n")
    f.write("#   1 - for MAIN-menu,  0 - Auto RUN\n")
    f.write("#\n")
    f.write("RUN=" + str(selfrun_mode) + "\n")
    f.write("#\n")
    f.write("# Directory for DATA:\n")
    f.write("#\n")
    f.write(selfpathfile + '\n')
    f.write("#\n")
    f.write("# AE33:   IP address and Port:\n")
    f.write("#\n")
    f.write("IP=" + selfIPname + '  ' + str(selfPort) +"\n")
    f.write("#\n")
    f.write("# AE33:  Last Records:\n")
    f.write("#\n")
    f.write("MINID=" + str(selfMAXID) + "\n")
    f.write("#\n")
    f.close()

write_path_file()

In [ ]:
import datetime
from datetime import datetime

lasttime = '2021/11/17 00:24:00'
nowtime  = '2021/11/17 00:17:00'

#dt = datetime.timedelta(lasttime-nowtime)
#line_db = '2011-11-04 00:05:23.283'

lasttime = datetime.strptime(lasttime, "%Y/%m/%d %H:%M:%S")
nowtime  = datetime.strptime(nowtime,  "%Y/%m/%d %H:%M:%S")
print(nowtime - lasttime)
if nowtime <= lasttime:
    print("no, ranshe")
else:
    print('yes')

-1 day, 23:53:00
no, ranshe


In [ ]:
selfbuff="AE33>AE33-S08-01006|934803|11/25/2021 4:14:00 PM|11/25/2021 4:15:00 PM|15|4/26/2020 5:56:15 PM|936036|296054|640942|915985|356924|669352|914645|390507|674155|909862|512676|794568|911313|513404|795643|806816|576523|835445|882069|599657|851611|282|380|487|322|436|530|335|410|528|301|321|429|323|370|461|328|322|420|432|342|359|0.004438983|0.005176876|0.005624666|0.006049186|0.006050732|0.005916851|0.006023637|22.25969|101325.0|21.1|3787|1214|5001|31.0|38.0|33.0|0|10|10|0|0|1014|150|0|29682|15\
AE33>AE33-S08-01006|934804|12/25/2021 4:15:00 PM|11/25/2021 4:16:00 PM|15|4/26/2020 5:56:15 PM|935996|295934|640817|915960|356794|669247|914634|390397|674071|909774|512509|794419|911268|513267|795532|806804|576410|835384|882018|599523|851516|296|387|512|349|418|574|309|402|488|298|368|425|317|409|452|351|351|450|439|347|351|0.004438719|0.005175995|0.005624828|0.006049688|0.006052002|0.005914216|0.006021341|30.41262|101325.0|21.1|3785|1216|5001|31.0|41.0|33.0|0|10|10|0|0|1014|150|0|29682|15\
AE33>AE33-S08-01006|934804|12/25/2021 5:15:00 PM|11/25/2021 4:16:00 PM|15|4/26/2020 5:56:15 PM|935996|295934|640817|915960|356794|669247|914634|390397|674071|909774|512509|794419|911268|513267|795532|806804|576410|835384|882018|599523|851516|296|387|512|349|418|574|309|402|488|298|368|425|317|409|452|351|351|450|439|347|351|0.004438719|0.005175995|0.005624828|0.006049688|0.006052002|0.005914216|0.006021341|30.41262|101325.0|21.1|3785|1216|5001|31.0|41.0|33.0|0|10|10|0|0|1014|150|0|29682|15\
AE33>AE33-S08-01006|934804|12/25/2021 6:15:00 PM|11/25/2021 4:16:00 PM|15|4/26/2020 5:56:15 PM|935996|295934|640817|915960|356794|669247|914634|390397|674071|909774|512509|794419|911268|513267|795532|806804|576410|835384|882018|599523|851516|296|387|512|349|418|574|309|402|488|298|368|425|317|409|452|351|351|450|439|347|351|0.004438719|0.005175995|0.005624828|0.006049688|0.006052002|0.005914216|0.006021341|30.41262|101325.0|21.1|3785|1216|5001|31.0|41.0|33.0|0|10|10|0|0|1014|150|0|29682|15\
AE33>AE33-S08-01006|934804|12/25/2021 7:15:00 PM|11/25/2021 4:16:00 PM|15|4/26/2020 5:56:15 PM|935996|295934|640817|915960|356794|669247|914634|390397|674071|909774|512509|794419|911268|513267|795532|806804|576410|835384|882018|599523|851516|296|387|512|349|418|574|309|402|488|298|368|425|317|409|452|351|351|450|439|347|351|0.004438719|0.005175995|0.005624828|0.006049688|0.006052002|0.005914216|0.006021341|30.41262|101325.0|21.1|3785|1216|5001|31.0|41.0|33.0|0|10|10|0|0|1014|150|0|29682|15\
AE33>"
#print(buff)
#len(buff.split("|"))
selfbuff = selfbuff.split("AE33>")
#print(buff)

selfmm = 11
selfyy = 2021
selffilename_raw = ''
selffile_raw = None
for line in selfbuff:
    if len(line) < 50:
        continue

    mm, dd, yy = line.split("|")[2][:10].split('/')
    if mm != selfmm or yy != selfyy:
        filename = '_'.join((yy, mm)) + '_AE33-S08-01006.raw'
        if selffile_raw:
            selffile_raw.close()
        selffile_raw = open(filename, "a")
    selffile_raw.write(line)
    selffile_raw.write('\n')
    
selffile_raw.flush()
selfmm = mm
selfyy = yy

In [ ]:
def fill_header():
    selffile_header = "AETHALOMETER\nSerial number = AE33-S08-01006\nApplication version = 1.6.7.0\nNumber of channels = 7\n\nDate(yyyy/MM/dd); Time(hh:mm:ss); Timebase; RefCh1; Sen1Ch1; Sen2Ch1; RefCh2; Sen1Ch2; Sen2Ch2; RefCh3; Sen1Ch3; Sen2Ch3; RefCh4; Sen1Ch4; Sen2Ch4; RefCh5; Sen1Ch5; Sen2Ch5; RefCh6; Sen1Ch6; Sen2Ch6; RefCh7; Sen1Ch7; Sen2Ch7; Flow1; Flow2; FlowC; Pressure(Pa); Temperature(°C); BB(%); ContTemp; SupplyTemp; Status; ContStatus; DetectStatus; LedStatus; ValveStatus; LedTemp; BC11; BC12; BC1; BC21; BC22; BC2; BC31; BC32; BC3; BC41; BC42; BC4; BC51; BC52; BC5; BC61; BC62; BC6; BC71; BC72; BC7; K1; K2; K3; K4; K5; K6; K7; TapeAdvCount;\n\n\n"
    print(selffile_header)

fill_header()


AETHALOMETER
Serial number = AE33-S08-01006
Application version = 1.6.7.0
Number of channels = 7

Date(yyyy/MM/dd); Time(hh:mm:ss); Timebase; RefCh1; Sen1Ch1; Sen2Ch1; RefCh2; Sen1Ch2; Sen2Ch2; RefCh3; Sen1Ch3; Sen2Ch3; RefCh4; Sen1Ch4; Sen2Ch4; RefCh5; Sen1Ch5; Sen2Ch5; RefCh6; Sen1Ch6; Sen2Ch6; RefCh7; Sen1Ch7; Sen2Ch7; Flow1; Flow2; FlowC; Pressure(Pa); Temperature(°C); BB(%); ContTemp; SupplyTemp; Status; ContStatus; DetectStatus; LedStatus; ValveStatus; LedTemp; BC11; BC12; BC1; BC21; BC22; BC2; BC31; BC32; BC3; BC41; BC42; BC4; BC51; BC52; BC5; BC61; BC62; BC6; BC71; BC72; BC7; K1; K2; K3; K4; K5; K6; K7; TapeAdvCount;





In [ ]:
columns = 'Date(yyyy/MM/dd); Time(hh:mm:ss); Timebase; RefCh1; Sen1Ch1; Sen2Ch1; RefCh2; Sen1Ch2; Sen2Ch2; RefCh3; Sen1Ch3; Sen2Ch3; RefCh4; Sen1Ch4; Sen2Ch4; RefCh5; Sen1Ch5; Sen2Ch5; RefCh6; Sen1Ch6; Sen2Ch6; RefCh7; Sen1Ch7; Sen2Ch7; Flow1; Flow2; FlowC; Pressure(Pa); Temperature(°C); BB(%); ContTemp; SupplyTemp; Status; ContStatus; DetectStatus; LedStatus; ValveStatus; LedTemp; BC11; BC12; BC1; BC21; BC22; BC2; BC31; BC32; BC3; BC41; BC42; BC4; BC51; BC52; BC5; BC61; BC62; BC6; BC71; BC72; BC7; K1; K2; K3; K4; K5; K6; K7; TapeAdvCount;'
columns = columns[:-1].split('; ')
print(len(columns),columns)

67 ['Date(yyyy/MM/dd)', 'Time(hh:mm:ss)', 'Timebase', 'RefCh1', 'Sen1Ch1', 'Sen2Ch1', 'RefCh2', 'Sen1Ch2', 'Sen2Ch2', 'RefCh3', 'Sen1Ch3', 'Sen2Ch3', 'RefCh4', 'Sen1Ch4', 'Sen2Ch4', 'RefCh5', 'Sen1Ch5', 'Sen2Ch5', 'RefCh6', 'Sen1Ch6', 'Sen2Ch6', 'RefCh7', 'Sen1Ch7', 'Sen2Ch7', 'Flow1', 'Flow2', 'FlowC', 'Pressure(Pa)', 'Temperature(°C)', 'BB(%)', 'ContTemp', 'SupplyTemp', 'Status', 'ContStatus', 'DetectStatus', 'LedStatus', 'ValveStatus', 'LedTemp', 'BC11', 'BC12', 'BC1', 'BC21', 'BC22', 'BC2', 'BC31', 'BC32', 'BC3', 'BC41', 'BC42', 'BC4', 'BC51', 'BC52', 'BC5', 'BC61', 'BC62', 'BC6', 'BC71', 'BC72', 'BC7', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'TapeAdvCount']


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=8ca19116-6b96-4631-b7d9-685d9c5feb9e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>